# ETL

## Librerías utilizadas

In [2]:
import pandas as pd
import numpy as np
import ast # Librería para generar funciones que permitan transformar los datos anidados
from datetime import date

## Lectura de archivo .csv

In [3]:
movies = pd.read_csv("./movies_dataset.csv")

C:\Users\Usuario\AppData\Local\Temp\ipykernel_2496\3827546478.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv("./movies_dataset.csv")


## Funciones utilizadas para los datos anidados

In [4]:
'''
    Funciones que permiten extraer los datos anidados de los campos: genres(los generos asociados a las peliculas), 
    production_companies(compañias productoras), production_countries(paises donde se produjerón las peliculas), 
    spoken_languages(idiomas que se hablan en las peliculas).
'''

'''
Función que permite convertir los campos anidados en listas
'''
def convertir(obj):
    if isinstance(obj, str) and '{' in obj:
        cL = []
        for i in  ast.literal_eval(obj):
            cL.append(i['name']);
        return cL

    
'''
    Función que va a permitir extraer los datos de las franquicias o series a la que pertenece 
    la pelicula (belongs_to_collection), guardada como diccionario.
'''
def convertir2(obj):
    if isinstance(obj, str) and '{' in obj:
        dic = ast.literal_eval(obj)
        return dic['name']

 
'''
    Función que permite converir los días de la semanas
'''   
def dia_sem(clave):
    dia = {1:'Lunes', 2:'Martes', 3:'Miércoles', 4:'Jueves', 5:'Viernes', 6:'Sábado', 7:'Domingo'}
    return dia[clave]

'''
    Función que permite converir los meses númericos del año a formato texto
'''   
def meses(clave):
    mesesA = {1:'Enero', 2:'Febrero', 3:'Marzo', 4:'Abril', 5:'Mayo', 6:'Junio', 7:'Julio', 8:'Agosto', 9:'Septiembre', 10:'Octubre', 11:'Noviembre',12:'Diciembre'}
    return mesesA[clave]


## Transformar datos anidados

In [5]:
# Transformación de los datos anidados
movies['genres'] = movies['genres'].apply(convertir) # Generos de las peliculas
movies['production_companies'] = movies['production_companies'].apply(convertir) # Compañias productoras
movies['belongs_to_collection'] = movies['belongs_to_collection'].apply(convertir2) # Franquicias
movies['production_countries'] = movies['production_countries'].apply(convertir) # Paises productores
movies['spoken_languages'] = movies['spoken_languages'].apply(convertir) # Idiomas de las peliculas

In [6]:
# Rellenar espacios en blancos de paises y compañias para que las funciones trabajen bien
movies['production_companies'] = movies['production_companies'].fillna('Sin datos')
movies['production_countries'] = movies['production_countries'].fillna('Sin datos')

## Rellenar valores nulos de "revenue y budget" con ceros

In [7]:
# Convierte el presupuesto de la pelicula (budget) en float
movies['budget'] = pd.to_numeric(movies['budget'], errors='coerce')

# Reemplaza los valores nulos de "revenue y Budget" con 0
movies[['budget', 'revenue']] = movies[['budget', 'revenue']].fillna(0)

## Cambiar formato de fecha y eliminar valores nulos

In [8]:
# Se cambia el tipo objeto por tipo fecha
movies['release_date'] = pd.to_datetime(movies['release_date'], format='%Y-%m-%d', errors='coerce')

# Elina las fechas con valor nulo y se resetean los indices
movies = movies.dropna(subset=['release_date']).reset_index(drop=True)

## Crear columna de año de la fecha de estreno (release_year) 

In [9]:
# Crear columna release_year
movies['release_year'] = pd.DatetimeIndex(movies['release_date']).year

# Se crear una columna para almacenar el mes (release_month)
movies['release_month'] = pd.DatetimeIndex(movies['release_date']).month
# Se le aplica la función meses para convertir los números de los meses en letras
movies['release_month'] = movies['release_month'].apply(meses)

# Se crea una columna día (release_day)
movies['release_day'] = pd.DatetimeIndex(movies['release_date']).day
# Se genera un for para convertir los días de la semana en letras
i = 0
for i in range(len(movies)):
    numero_dia = date.isoweekday(movies['release_date'][i])
    movies['release_day'][i]= dia_sem(numero_dia)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_2496\1475138946.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['release_day'][i]= dia_sem(numero_dia)


## Crear columna retorno de inversión (return)

In [10]:
'''
    Se crea la columna 'return' mediante un where en numpy que nos permite 
    colocar en cero el resultado de return si revenue o budget tienen valor 0. 
'''
movies['return'] = np.where((movies['budget']==0) | (movies['revenue']==0), 0, movies['revenue']/movies['budget'])

## Eliminar columnas requeridas

In [11]:
# Se genera nuevamente la data sin las columnas especificadas en el .drop
movies = movies.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'vote_count', 'poster_path', 'homepage'])

In [12]:
movies.head()

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,runtime,spoken_languages,status,tagline,title,vote_average,release_year,release_month,release_day,return
0,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,...,81.0,[English],Released,NaN,Toy Story,7.7,1995,Octubre,Lunes,12.451801
1,None,65000000.0,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,...,104.0,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,1995,Diciembre,Viernes,4.043035
2,Grumpy Old Men Collection,0.0,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,...,101.0,[English],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,1995,Diciembre,Viernes,0.000000
3,None,16000000.0,"[Comedy, Drama, Romance]",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[Twentieth Century Fox Film Corporation],[United States of America],1995-12-22,...,127.0,[English],Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,1995,Diciembre,Viernes,5.090760
4,Father of the Bride Collection,0.0,[Comedy],11862,en,Just when George Banks has recovered from his ...,8.387519,"[Sandollar Productions, Touchstone Pictures]",[United States of America],1995-02-10,...,106.0,[English],Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,1995,Febrero,Viernes,0.000000


## Copia del dataset ya modificado

In [13]:
movies.reset_index().to_csv('mv_dat_mod.csv', header = True, index=False)